In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.optim.adam import Adam
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from tasks.wsd_gloss import enclose_keyword


In [3]:
def gloss_dfs(df):
    """convert rows in dataframe to GlossBERT format
    """
    df['enclosed_quotation'] = df.apply(enclose_keyword, axis=1)
    df_gl =  df[['enclosed_quotation','definition','label']]
    return df_gl[['enclosed_quotation','label']],df_gl[['definition','label']].drop_duplicates()
    


In [4]:
lemma,pos='machine','NN'
df_quotations = pd.read_pickle(f'./data/sfrel_quotations_{lemma}_{pos}.pickle')
df_quotations = df_quotations[~df_quotations.keyword_offset.isnull()]
df_quotations = df_quotations[~df_quotations.definition.isnull()].reset_index(drop=True)


In [6]:
print(df_quotations.shape)          
quotation_valuecounts = df_quotations.sense_id.value_counts()
df_quotations = df_quotations[df_quotations.sense_id.isin(quotation_valuecounts[quotation_valuecounts>1].index)]
print(df_quotations.shape)

(29752, 14)
(29321, 14)


In [7]:
#context_df,gloss_df = gloss_dfs((df_quotations))

KeyError: "['label'] not in index"

In [ ]:
print(context_df.shape,gloss_df.shape)

In [ ]:
train_data_path = Path("./data/training_data_ms")
train_data_path.mkdir(exist_ok=True)



In [48]:
for n, df in [('context',context_df),('gloss',gloss_df)]:

    df_out_path = train_data_path / f'{lemma}_{pos}_{n}'
    df_out_path.mkdir(exist_ok=True)

    df_train, df_test = train_test_split(df, 
                                        test_size=0.2, 
                                        random_state=42,
                                        shuffle=True,
                                        stratify=df[['label']]
                                        ) # 1st
                                                
    df_train, df_val = train_test_split(df_train, 
                                        test_size=0.1, 
                                        random_state=42,
                                        shuffle=True,
                                        stratify=df_train[['label']] # bug here, try to do the stratification better
                                        ) # 2nd
    
    df_train.to_csv(df_out_path / 'train.csv', index = False, sep='\t')  
    df_val.to_csv(df_out_path / 'dev.csv', index = False, sep='\t')  
    df_test.to_csv(df_out_path / 'test.csv', index = False, sep='\t')

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.